In [2]:
import os
import time
import psycopg2
AWS_REGION = 'us-east-1'
AWS_PROFILE = 'localstack'
ENDPOINT_URL = "http://localhost:4566"


In [3]:
import boto3
boto3.setup_default_session(profile_name=AWS_PROFILE)
sqs = boto3.client('sqs',region_name='us-east-1',endpoint_url = ENDPOINT_URL)
queue_url= 'http://localhost:4566/000000000000/login-queue'

In [4]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="postgres")

In [5]:
cur = conn.cursor()
print('PostgreSQL database version:')
cur.execute('SELECT version()')
db_version = cur.fetchone()
print(db_version)
alter_sql = """Alter table user_logins alter column app_version type varchar(32)"""
cur.execute(alter_sql)
conn.commit()


PostgreSQL database version:
('PostgreSQL 10.21 (Debian 10.21-1.pgdg90+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 6.3.0-18+deb9u1) 6.3.0 20170516, 64-bit',)


In [6]:
cur.execute("""
    SELECT column_name, data_type, character_maximum_length
    FROM information_schema.columns
    WHERE table_name = 'user_logins'
""")
print(cur.fetchall())

[('user_id', 'character varying', 128), ('device_type', 'character varying', 32), ('masked_ip', 'character varying', 256), ('masked_device_id', 'character varying', 256), ('locale', 'character varying', 32), ('app_version', 'character varying', 32), ('create_date', 'date', None)]


In [7]:
import json
from datetime import datetime
import hashlib

def mask_fields(data, fields_to_mask):
    for field in fields_to_mask:
        if field in data:
            # Generate a hash of the original value
            hashed_value = hashlib.sha256(data[field].encode()).hexdigest()
            # Keep the first 8 characters of the hash
            data[field] = hashed_value[:8]
    return data


c=2
while c>1:
    response = sqs.receive_message(QueueUrl=queue_url,AttributeNames=['SentTimestamp'],
    MaxNumberOfMessages=1,
    VisibilityTimeout=0,
    WaitTimeSeconds=10
    )
    if 'Messages' in response:
        messages = response['Messages']

        for message in messages:
            receiptHandle = message['ReceiptHandle']
            body = json.loads(message['Body'])
            print(receiptHandle)
            print(body)
            print(mask_fields(body,['device_id', 'ip']))
            if 'user_id' in body.keys():
                value = (body['user_id'],body['device_type'],body['ip'],body['device_id'],body['locale'],body['app_version'],str(datetime.now()))
                sql = f"INSERT INTO user_logins (user_id,device_type,masked_ip,masked_device_id,locale,app_version,create_date) VALUES (%s,%s,%s,%s,%s,%s,%s)"
                print(sql,value)
                try:
                    cur.execute(sql,value)
                    conn.commit()
                    sqs.delete_message(QueueUrl=queue_url,ReceiptHandle=receiptHandle)
                except(Exception, psycopg2.DatabaseError) as error:
                    print(error)
            else:
                print('Invalid data')
                sqs.delete_message(QueueUrl=queue_url,ReceiptHandle=receiptHandle)
    else:
        c-=1

In [8]:
import pandas as pd
cur.execute("select * from user_logins")
data = cur.fetchall()
df = pd.DataFrame(data, columns =['user_id','device_type','masked_ip','masked_device_id','locale','app_version','create_date'])
print(df.shape)
print(df.head())

(99, 7)
                                user_id device_type masked_ip  \
0  424cdd21-063a-43a7-b91b-7ca1a833afae     android  a6d0e2f2   
1  c0173198-76a8-4e67-bfc2-74eaa3bbff57         ios  7b03f7d7   
2  66e0635b-ce36-4ec7-aa9e-8a8fca9b83d4         ios  fa7fca28   
3  181452ad-20c3-4e93-86ad-1934c9248903     android  b21d1c92   
4  60b9441c-e39d-406f-bba0-c7ff0e0ee07f     android  587f5a11   

  masked_device_id locale app_version create_date  
0         4f00c1a8     RU       2.3.0  2023-01-28  
1         a857e702     PH       0.2.6  2023-01-28  
2         e84fb3e1   None       2.2.1  2023-01-28  
3         94b571f6     ID        0.96  2023-01-28  
4         3102ec6d     FR       0.4.6  2023-01-28  
